# 🎯 Vehicle-lang Human-in-the-Loop Hyperrectangle Generator

This notebook provides an interactive interface for creating hyperrectangles that define attack boundaries for adversarial training in the NIDS system.

## Features:
- **Data-driven suggestions**: Analyze training data to suggest initial bounds
- **Interactive refinement**: Human expert can adjust and validate bounds
- **Vehicle-lang integration**: Generate formal specifications for verification
- **Visualization**: See feature distributions and attack patterns
- **Export**: Ready-to-use hyperrectangles for adversarial training

In [ ]:
import sys
import os
sys.path.append('/home/gugr/NIDS-Verify-Personal')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

from src.tools.vehicle_hyperrectangles import (
    VehicleHyperrectangleGenerator, 
    Hyperrectangle, 
    FeatureBounds,
    create_example_hyperrectangles
)

# Configure plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

print("🚀 Vehicle-lang Hyperrectangle Generator loaded!")

## 1. Initialize Generator and Load Data

In [ ]:
# Initialize the generator
generator = VehicleHyperrectangleGenerator(
    data_path="data/preprocessed-dos-train.csv",
    output_dir="vehicle_output"
)

# Load and analyze data
data = generator.load_data()
print(f"📊 Loaded {len(data)} samples with {len(generator.feature_names)} features")
print(f"📈 Attack samples: {len(data[data['label'] == 1])}")
print(f"📉 Benign samples: {len(data[data['label'] == 0])}")

# Display basic statistics
display(data.describe())

## 2. Analyze Feature Distributions

In [ ]:
# Analyze feature distributions
analysis = generator.analyze_feature_distributions()

# Create visualization
top_features = generator.visualize_feature_distributions(top_features=10)

# Display top separating features
print("🎯 Top 10 Features by Separation Score:")
for i, feature in enumerate(top_features, 1):
    score = analysis[feature]['separation_score']
    print(f"{i:2d}. {feature:<30} (Score: {score:.3f})")

## 3. Interactive Hyperrectangle Creation

Use the widgets below to create hyperrectangles interactively:

In [ ]:
# Create interactive widgets for hyperrectangle creation
class InteractiveHyperrectangleCreator:
    def __init__(self, generator):
        self.generator = generator
        self.current_hyperrectangle = None
        
    def create_widgets(self):
        # Basic info widgets
        self.name_widget = widgets.Text(
            value='My Attack Boundary',
            description='Name:',
            style={'description_width': 'initial'}
        )
        
        self.description_widget = widgets.Textarea(
            value='Description of the attack pattern',
            description='Description:',
            style={'description_width': 'initial'}
        )
        
        self.attack_type_widget = widgets.Dropdown(
            options=['DoS', 'DDoS', 'PortScan', 'BruteForce', 'Intrusion', 'Other'],
            value='DoS',
            description='Attack Type:',
            style={'description_width': 'initial'}
        )
        
        self.method_widget = widgets.Dropdown(
            options=['percentile', 'iqr', 'mean_std', 'min_max'],
            value='percentile',
            description='Suggestion Method:',
            style={'description_width': 'initial'}
        )
        
        self.confidence_widget = widgets.FloatSlider(
            value=0.8,
            min=0.0,
            max=1.0,
            step=0.1,
            description='Confidence:',
            style={'description_width': 'initial'}
        )
        
        # Button to generate suggestions
        self.suggest_button = widgets.Button(
            description='Generate Suggestions',
            button_style='info'
        )
        self.suggest_button.on_click(self.generate_suggestions)
        
        # Container for feature bound widgets (will be populated after suggestions)
        self.bounds_container = widgets.VBox()
        
        # Create and validate buttons
        self.create_button = widgets.Button(
            description='Create Hyperrectangle',
            button_style='success'
        )
        self.create_button.on_click(self.create_hyperrectangle)
        
        return widgets.VBox([
            widgets.HTML('<h3>🎯 Create New Hyperrectangle</h3>'),
            self.name_widget,
            self.description_widget,
            self.attack_type_widget,
            self.method_widget,
            self.confidence_widget,
            self.suggest_button,
            self.bounds_container,
            self.create_button
        ])
    
    def generate_suggestions(self, button):
        """Generate suggested bounds based on data analysis"""
        suggested_bounds = self.generator.suggest_hyperrectangle_bounds(
            attack_type=self.attack_type_widget.value,
            method=self.method_widget.value
        )
        
        # Create widgets for each feature bound
        bound_widgets = []
        self.bound_widgets_list = []  # Store references
        
        for i, bound in enumerate(suggested_bounds[:10]):  # Limit to top 10 features
            min_widget = widgets.FloatText(
                value=bound.min_value,
                description=f'{bound.name} Min:',
                style={'description_width': '200px'},
                layout=widgets.Layout(width='300px')
            )
            
            max_widget = widgets.FloatText(
                value=bound.max_value,
                description=f'{bound.name} Max:',
                style={'description_width': '200px'},
                layout=widgets.Layout(width='300px')
            )
            
            include_widget = widgets.Checkbox(
                value=True,
                description='Include',
                style={'description_width': 'initial'}
            )
            
            bound_box = widgets.HBox([
                include_widget,
                min_widget,
                max_widget
            ])
            
            bound_widgets.append(bound_box)
            self.bound_widgets_list.append({
                'bound': bound,
                'include': include_widget,
                'min': min_widget,
                'max': max_widget
            })
        
        self.bounds_container.children = [
            widgets.HTML('<h4>📊 Adjust Feature Bounds</h4>')
        ] + bound_widgets
    
    def create_hyperrectangle(self, button):
        """Create hyperrectangle from widget values"""
        if not hasattr(self, 'bound_widgets_list'):
            print("⚠️ Please generate suggestions first!")
            return
        
        # Collect selected bounds
        final_bounds = []
        for widget_data in self.bound_widgets_list:
            if widget_data['include'].value:
                bound = FeatureBounds(
                    name=widget_data['bound'].name,
                    min_value=widget_data['min'].value,
                    max_value=widget_data['max'].value,
                    description=widget_data['bound'].description,
                    unit=widget_data['bound'].unit
                )
                final_bounds.append(bound)
        
        # Create hyperrectangle
        hyperrectangle = Hyperrectangle(
            name=self.name_widget.value,
            description=self.description_widget.value,
            feature_bounds=final_bounds,
            attack_type=self.attack_type_widget.value,
            confidence=self.confidence_widget.value
        )
        
        # Add to generator
        self.generator.add_hyperrectangle(hyperrectangle)
        self.current_hyperrectangle = hyperrectangle
        
        print(f"✅ Created hyperrectangle '{hyperrectangle.name}' with {len(final_bounds)} features")
        print(f"📐 Volume: {hyperrectangle.volume():.6f}")
        
        # Clear form for next hyperrectangle
        self.bounds_container.children = []

# Create the interactive creator
creator = InteractiveHyperrectangleCreator(generator)
display(creator.create_widgets())

## 4. Create Example Hyperrectangles

If you want to start with some example hyperrectangles:

In [ ]:
# Create example hyperrectangles
examples = create_example_hyperrectangles(generator)

print("📋 Created example hyperrectangles:")
for example in examples:
    generator.add_hyperrectangle(example)
    print(f"  • {example.name} ({example.attack_type}) - {len(example.feature_bounds)} features")
    print(f"    Volume: {example.volume():.6f}, Confidence: {example.confidence}")

## 5. Validate Hyperrectangles

In [ ]:
# Validate hyperrectangles against training data
if generator.hyperrectangles:
    validation_results = generator.validate_hyperrectangles()
    
    # Create validation summary
    validation_df = pd.DataFrame(validation_results).T
    
    print("📊 Hyperrectangle Validation Results:")
    display(validation_df)
    
    # Plot validation metrics
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Precision vs Recall
    axes[0,0].scatter(validation_df['recall'], validation_df['precision'], 
                      s=validation_df['confidence']*100, alpha=0.7)
    axes[0,0].set_xlabel('Recall')
    axes[0,0].set_ylabel('Precision')
    axes[0,0].set_title('Precision vs Recall')
    axes[0,0].grid(True, alpha=0.3)
    
    # Volume vs Confidence
    axes[0,1].scatter(validation_df['volume'], validation_df['confidence'], alpha=0.7)
    axes[0,1].set_xlabel('Volume')
    axes[0,1].set_ylabel('Confidence')
    axes[0,1].set_title('Volume vs Confidence')
    axes[0,1].grid(True, alpha=0.3)
    
    # Sample coverage
    validation_df['total_samples_in_rect'].plot(kind='bar', ax=axes[1,0])
    axes[1,0].set_title('Total Samples in Each Hyperrectangle')
    axes[1,0].set_ylabel('Sample Count')
    
    # Attack vs Benign samples
    validation_df[['attack_samples_in_rect', 'benign_samples_in_rect']].plot(kind='bar', ax=axes[1,1])
    axes[1,1].set_title('Attack vs Benign Samples')
    axes[1,1].set_ylabel('Sample Count')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("⚠️ No hyperrectangles created yet. Create some using the interactive tool above.")

## 6. Generate Vehicle-lang Specification

In [ ]:
if generator.hyperrectangles:
    # Generate Vehicle specification
    spec_file = generator.generate_vehicle_specification("nids_attack_boundaries")
    
    print(f"📝 Generated Vehicle specification: {spec_file}")
    
    # Display the specification
    with open(spec_file, 'r') as f:
        spec_content = f.read()
    
    print("\n📋 Vehicle-lang Specification:")
    print("=" * 50)
    print(spec_content[:2000] + "..." if len(spec_content) > 2000 else spec_content)
    
else:
    print("⚠️ No hyperrectangles to generate specification from.")

## 7. Export for Adversarial Training

In [ ]:
if generator.hyperrectangles:
    print("🎯 EXPORTING FOR ADVERSARIAL TRAINING")
    print("=" * 50)
    
    # Export perturbation bounds for PGD adversarial training
    perturbation_bounds = generator.export_for_adversarial_training()
    
    print(f"✅ Exported {len(perturbation_bounds)} perturbation bounds for adversarial training")
    print("\n📋 IMPORTANT DISTINCTION:")
    print("• Hyperrectangles define ATTACK REGIONS (absolute feature ranges)")
    print("• PGD needs PERTURBATION BOUNDS (relative changes from clean examples)")
    print("• This tool converts attack regions → perturbation bounds automatically")
    
    # Show the perturbation bounds
    print(f"\n📊 Perturbation Bounds (for PGD):")
    for i, bounds in enumerate(perturbation_bounds[:10]):  # Show first 10
        print(f"  Feature {i+1}: [{bounds[0]:.3f}, {bounds[1]:.3f}]")
    if len(perturbation_bounds) > 10:
        print(f"  ... and {len(perturbation_bounds)-10} more features")
    
    # Show usage examples
    print(f"\n🚀 USAGE IN ADVERSARIAL TRAINING:")
    print("```python")
    print("# Load the perturbation bounds (NOT attack regions)")
    print("perturbation_bounds = np.load('vehicle_output/perturbation_bounds.npy')")
    print("")
    print("# Use with your PGD implementation")
    print("trainer = NIDSTrainer(model)")
    print("history = trainer.train(train_ds, val_ds, 'adversarial', perturbation_bounds)")
    print("```")
    
    # Save session for later use
    generator.save_session()
    print("\n💾 Session saved for future use")
    
else:
    print("⚠️ No hyperrectangles to export.")

## 8. Understanding Attack Regions vs Perturbation Bounds

### 🔍 **Important Distinction**

This tool generates **attack region hyperrectangles** but your PGD adversarial training needs **perturbation bounds**. Here's the difference:

#### 📐 **Attack Regions (Hyperrectangles)**
- **What**: Absolute feature value ranges where attacks occur
- **Example**: "DoS attacks have packet_size ∈ [0.1, 0.3] and inter_arrival_time ∈ [0.0, 0.1]"
- **Used for**: 
  - Formal verification with Vehicle-lang
  - Understanding attack patterns
  - Security analysis

#### ⚡ **Perturbation Bounds**
- **What**: Maximum allowed changes from clean examples
- **Example**: "Can perturb packet_size by ±0.15 and inter_arrival_time by ±0.05"
- **Used for**:
  - PGD adversarial training
  - Generating adversarial examples

### 🔄 **Automatic Conversion**

The tool automatically converts attack regions → perturbation bounds:

```python
# Attack region: DoS packets have size ∈ [0.1, 0.3]
# Clean traffic baseline: ~0.5
# Max perturbation needed: max(0.5-0.1, 0.3-0.5) = 0.4
# Symmetric bound: [-0.4, +0.4] (clamped to reasonable limits)
```

### ✅ **What You Should Use**

- **For adversarial training**: Use `perturbation_bounds.npy`
- **For verification**: Use `attack_regions.npy` + Vehicle-lang spec

In [ ]:
# Display summary
print("🎊 HYPERRECTANGLE GENERATION COMPLETE!")
print("=" * 50)
print(f"📊 Total hyperrectangles created: {len(generator.hyperrectangles)}")

if generator.hyperrectangles:
    attack_types = [rect.attack_type for rect in generator.hyperrectangles]
    unique_types = list(set(attack_types))
    print(f"🎯 Attack types covered: {', '.join(unique_types)}")
    
    avg_confidence = np.mean([rect.confidence for rect in generator.hyperrectangles])
    print(f"🎯 Average confidence: {avg_confidence:.2f}")
    
    total_volume = sum([rect.volume() for rect in generator.hyperrectangles])
    print(f"📐 Total coverage volume: {total_volume:.6f}")

print("\n? IMPORTANT: Two Types of Bounds Generated")
print("1. 📐 ATTACK REGIONS: Absolute feature ranges where attacks occur")
print("   - Used for: Verification, analysis, understanding attack patterns")
print("   - File: vehicle_output/attack_regions.npy")
print("")
print("2. ⚡ PERTURBATION BOUNDS: Relative changes from clean examples")
print("   - Used for: PGD adversarial training (what your code needs)")
print("   - File: vehicle_output/perturbation_bounds.npy")

print("\n?🚀 Next Steps:")
print("1. Review the generated Vehicle-lang specification")
print("2. Use the PERTURBATION bounds in adversarial training:")
print("   ```python")
print("   perturbation_bounds = np.load('vehicle_output/perturbation_bounds.npy')")
print("   trainer = NIDSTrainer(model)")
print("   history = trainer.train(train_ds, val_ds, 'adversarial', perturbation_bounds)")
print("   ```")
print("3. Use ATTACK REGIONS for verification with Vehicle-lang")
print("4. Iterate and refine based on training results")